## Ensembl to RefSeq Mapping

The constraint table from gnomAD has duplicate gene ID's - in the example of TUBB3 one gene ID is missannotated. Given out analysis is by transcript, it is probably better to use the transcript table from gnomAD. Howver, gnomAD used ENSEMBL transcripts and we used RefSeq Transcripts. Can map the two through biomart:

http://www.ensembl.org/biomart/martview/e81bf786e69482239d8e7799ec2c9e9e



In [19]:
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window


In [20]:
val customSchema = new StructType(Array(
  StructField("GeneID",StringType,true),
  StructField("GeneIDVer",StringType,true),
  StructField("TranscriptID",StringType,true),
  StructField("TranscriptIDVer",StringType,true),
  StructField("EnsemblGeneSymbol",StringType,true),
  StructField("GeneType",StringType,true),
  StructField("GENE",StringType,true),
  StructField("RefSeq",StringType,true),
  StructField("NCBIGeneID",IntegerType,true)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(GeneID,StringType,true), StructField(GeneIDVer,StringType,true), StructField(TranscriptID,StringType,true), StructField(TranscriptIDVer,StringType,true), StructField(EnsemblGeneSymbol,StringType,true), StructField(GeneType,StringType,true), StructField(GENE,StringType,true), StructField(RefSeq,StringType,true), StructField(NCBIGeneID,IntegerType,true))


In [21]:
val df_ens2ref = (spark.read
        .format("csv")
        .option("header", "true")
        .option("delimiter", "\t")
        .option("nullValues", "")
        .schema(customSchema)
        .load("s3://nch-igm-research-projects/rna_stability/peter/ensembl_2_refeq.txt")
             )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_ens2ref: org.apache.spark.sql.DataFrame = [GeneID: string, GeneIDVer: string ... 7 more fields]


In [22]:
df_ens2ref.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GeneID: string (nullable = true)
 |-- GeneIDVer: string (nullable = true)
 |-- TranscriptID: string (nullable = true)
 |-- TranscriptIDVer: string (nullable = true)
 |-- EnsemblGeneSymbol: string (nullable = true)
 |-- GeneType: string (nullable = true)
 |-- GENE: string (nullable = true)
 |-- RefSeq: string (nullable = true)
 |-- NCBIGeneID: integer (nullable = true)



In [23]:
df_ens2ref.filter($"GENE" === "TUBB3").show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+---------------+------------------+-----------------+--------------+-----+------------+----------+
|         GeneID|        GeneIDVer|   TranscriptID|   TranscriptIDVer|EnsemblGeneSymbol|      GeneType| GENE|      RefSeq|NCBIGeneID|
+---------------+-----------------+---------------+------------------+-----------------+--------------+-----+------------+----------+
|ENSG00000258947|ENSG00000258947.7|ENST00000555810| ENST00000555810.5|            TUBB3|protein_coding|TUBB3|        null|     10381|
|ENSG00000258947|ENSG00000258947.7|ENST00000554444| ENST00000554444.5|            TUBB3|protein_coding|TUBB3|NM_001197181|     10381|
|ENSG00000258947|ENSG00000258947.7|ENST00000556565| ENST00000556565.5|            TUBB3|protein_coding|TUBB3|        null|     10381|
|ENSG00000258947|ENSG00000258947.7|ENST00000315491|ENST00000315491.12|            TUBB3|protein_coding|TUBB3|   NM_006086|     10381|
|ENSG00000258947|ENSG00000258947.7|ENST00000553656| ENST000005

Double check there are no duplicates

In [24]:
val df_ref2ens = df_ens2ref.filter($"RefSeq".isNotNull).select("TranscriptID","RefSeq").sort($"TranscriptID").distinct

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_ref2ens: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [TranscriptID: string, RefSeq: string]


In [25]:
df_ref2ens.show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------+
|   TranscriptID|      RefSeq|
+---------------+------------+
|ENST00000000233|   NM_001662|
|ENST00000000412|   NM_002355|
|ENST00000000442|   NM_004451|
|ENST00000000442|NM_001282451|
|ENST00000001008|   NM_002014|
|ENST00000001146|   NM_019885|
|ENST00000002125|   NM_144736|
|ENST00000002125|NM_001350024|
|ENST00000002125|NM_001350025|
|ENST00000002165|   NM_032020|
|ENST00000002501|NM_001042610|
|ENST00000002596|   NM_005114|
|ENST00000002829|   NM_004186|
|ENST00000003084|   NM_000492|
|ENST00000003100|   NM_000786|
|ENST00000003302|NM_001346259|
|ENST00000003302|NM_001346257|
|ENST00000003302|NM_001346253|
|ENST00000003302|NM_001346258|
|ENST00000003302|   NM_020886|
+---------------+------------+
only showing top 20 rows



In [26]:
df_ref2ens.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res15: Long = 61733


In [27]:
df_ref2ens.select($"RefSeq").distinct.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res16: Long = 56317


In [28]:
df_ref2ens.groupBy($"RefSeq").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      RefSeq|count|
+------------+-----+
|NM_001242867|   25|
|   NM_006737|   25|
|   NM_153443|   23|
|NM_001080770|   22|
|   NM_015868|   21|
|   NM_014218|   19|
|   NM_002255|   19|
|NM_001145458|   17|
|NM_001145457|   16|
|NM_001242357|   16|
|   NM_004829|   16|
|NM_001281971|   15|
|NM_001281972|   15|
|NM_001080772|   14|
|NM_001322168|   14|
|   NM_012314|   13|
|   NM_002000|   11|
|   NM_133274|   11|
|   NM_133271|   11|
|   NM_133269|   11|
+------------+-----+
only showing top 20 rows



## gnomAD lof metrics by transcript

We want to link the data to gnomAD constraint metrics (LOEUF and pLI):

Supplemental Table describing data fields:

https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-020-2308-7/MediaObjects/41586_2020_2308_MOESM1_ESM.pdf

Select the following columns from the main file:

* **gene**: Gene name
* **gene_id**: Ensembl gene ID
* **transcript**: Ensembl transcript ID (Gencode v19)
* **obs_mis**: Number of observed missense variants in transcript
* **exp_mis**: Number of expected missense variants in transcript
* **oe_mis**: Observed over expected ratio for missense variants in transcript (obs_mis divided by exp_mis)
* **obs_syn**: Number of observed synonymous variants in transcript
* **exp_syn**: Number of expected synonymous variants in transcript
* **oe_syn**: Observed over expected ratio for missense variants in transcript (obs_syn divided by exp_syn)

* **p**: The estimated proportion of haplotypes with a pLoF variant. Defined as: 1 -sqrt(no_lofs / defined)
* **pLI**: Probability of loss-of-function intolerance; probability that transcript falls into distribution of haploinsufficient genes (~9% o/e pLoF ratio; computed from gnomAD data)

* **pRec**: Probability that transcript falls into distribution of recessive genes (~46% o/e pLoF ratio; computed from gnomAD data)
* **pNull**: Probability that transcript falls into distribution of unconstrained genes (~100% o/epLoF ratio; computed from gnomAD data)
* **oe_lof_upper**: LOEUF: upper bound of 90% confidence interval for o/e ratio for pLoF variants (lower values indicate more constrained)
* **oe_lof_upper_rank**: Transcript’s rank of LOEUF value compared to all transcripts (lower values indicate more constrained)
* **oe_lof_upper_bin**: Decile bin of LOEUF for given transcript (lower values indicate more constrained)

In [29]:
val cols_constraint = Seq("gene","gene_id","transcript","obs_mis","exp_mis","oe_mis","obs_syn","exp_syn","oe_syn",
                          "p","pLI","pRec","pNull","oe_lof_upper","oe_lof_upper_rank","oe_lof_upper_bin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cols_constraint: Seq[String] = List(gene, gene_id, transcript, obs_mis, exp_mis, oe_mis, obs_syn, exp_syn, oe_syn, p, pLI, pRec, pNull, oe_lof_upper, oe_lof_upper_rank, oe_lof_upper_bin)


In [30]:
val df_loft_import = (spark.read
        .format("csv")
        .option("header", "true")
        .option("delimiter", "\t")
        .option("inferSchema", "true")
        .option("nullValues", "NA")
        .option("nanValue=", "NA")
        .load("s3://nch-igm-research-projects/rna_stability/peter/gnomad.v2.1.1.lof_metrics.by_transcript.txt")
        .select(cols_constraint.map(col): _*)
             )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_loft_import: org.apache.spark.sql.DataFrame = [gene: string, gene_id: string ... 14 more fields]


In [31]:
val df_loft = (df_loft_import.withColumn("p",col("p").cast(DoubleType))
                    .withColumn("pLI",col("pLI").cast(DoubleType))
                    .withColumn("pRec",col("pRec").cast(DoubleType))
                    .withColumn("pNull",col("pNull").cast(DoubleType))
                    .withColumn("oe_lof_upper",col("oe_lof_upper").cast(DoubleType))
                    .withColumn("oe_lof_upper_rank",col("oe_lof_upper_rank").cast(IntegerType))
                    .withColumn("oe_lof_upper_bin",col("oe_lof_upper_bin").cast(IntegerType))
                    .withColumnRenamed("gene", "gnomadGeneSymbol")
                    .withColumnRenamed("gene_id", "gnomadGeneID")
                    .withColumnRenamed("transcript", "TranscriptID")               
             )   
df_loft.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_loft: org.apache.spark.sql.DataFrame = [gnomadGeneSymbol: string, gnomadGeneID: string ... 14 more fields]
root
 |-- gnomadGeneSymbol: string (nullable = true)
 |-- gnomadGeneID: string (nullable = true)
 |-- TranscriptID: string (nullable = true)
 |-- obs_mis: string (nullable = true)
 |-- exp_mis: string (nullable = true)
 |-- oe_mis: string (nullable = true)
 |-- obs_syn: string (nullable = true)
 |-- exp_syn: string (nullable = true)
 |-- oe_syn: string (nullable = true)
 |-- p: double (nullable = true)
 |-- pLI: double (nullable = true)
 |-- pRec: double (nullable = true)
 |-- pNull: double (nullable = true)
 |-- oe_lof_upper: double (nullable = true)
 |-- oe_lof_upper_rank: integer (nullable = true)
 |-- oe_lof_upper_bin: integer (nullable = true)



In [32]:
df_loft.filter($"gene" === "TUBB3").show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+----------+--------+---------+------------+-----------------+----------------+
|gnomadGeneSymbol|   gnomadGeneID|   TranscriptID|obs_mis|   exp_mis|    oe_mis|obs_syn|   exp_syn|    oe_syn|        p|       pLI|    pRec|    pNull|oe_lof_upper|oe_lof_upper_rank|oe_lof_upper_bin|
+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+----------+--------+---------+------------+-----------------+----------------+
|           TUBB3|ENSG00000258947|ENST00000304984|     67|2.5624e+02|2.6148e-01|    147|1.1855e+02|1.2400e+00|1.1932E-5|   0.81145| 0.18766| 8.854E-4|        0.48|            17235|               2|
|           TUBB3|ENSG00000258947|ENST00000315491|     77|3.0026e+02|2.5644e-01|    172|1.3880e+02|1.2392e+00|1.1932E-5|   0.96761|0.032378|9.5178E-6|       0.315|             9211|               1|
|    

Lets check that TranscriptID is not duplicated

In [33]:
df_loft.select($"TranscriptID").distinct.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res20: Long = 80950


In [34]:
df_loft.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res21: Long = 80950


In [35]:
df_loft.filter($"p".isNotNull && $"pLI" < 0.9).
        select("gnomadGeneSymbol","p","pLI","oe_lof_upper","oe_lof_upper_rank","oe_lof_upper_bin").sort($"pLI".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------+-------+------------+-----------------+----------------+
|gnomadGeneSymbol|        p|    pLI|oe_lof_upper|oe_lof_upper_rank|oe_lof_upper_bin|
+----------------+---------+-------+------------+-----------------+----------------+
|           AP2S1|      0.0|0.89998|       0.402|            13402|               1|
|           ELMO1|5.5669E-5|0.89997|       0.333|            10049|               1|
|           ELMO1|5.5669E-5|0.89997|       0.333|            10048|               1|
|           ELMO1|5.5669E-5|0.89997|       0.333|            10050|               1|
|          RANBP3|4.0869E-6|0.89994|       0.401|            13373|               1|
|            MEN1|      0.0| 0.8999|       0.402|            13398|               1|
|         RUNX1T1|3.9864E-6|0.89987|       0.402|            13374|               1|
|            OGDH|5.5668E-5|0.89984|       0.329|             9852|               1|
|            ETS1|7.9536E-6|0.89984|       0.388|            1275

## Join gnomAD lof table to RefSeq to Ensembl table

In [36]:
val df_loft_ref = (df_loft.as("df_loft")
                       .join(df_ref2ens.as("df_ref2ens"), $"df_loft.TranscriptID" === $"df_ref2ens.TranscriptID", "inner")
                       .drop($"df_ref2ens.TranscriptID"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_loft_ref: org.apache.spark.sql.DataFrame = [gnomadGeneSymbol: string, gnomadGeneID: string ... 15 more fields]


Note that the number of rows has now increased from 80,950 to 95,806 - this is becuase ensemble transcripts can map to multiple RefSeq transcripts and vice versa. We now need to make a table where the RefSeq field is not duplicated.

In [37]:
df_loft_ref.groupBy($"RefSeq").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      RefSeq|count|
+------------+-----+
|NM_001190455|    2|
|   NM_032862|    2|
|NM_001122631|    2|
|   NM_206955|    2|
|   NM_145021|    2|
|NM_001291304|    2|
|NM_001289026|    2|
|NM_001009606|    2|
|   NM_133262|    2|
|NM_001289759|    2|
|NM_001122630|    2|
|NM_001291717|    2|
|NM_001284499|    2|
|NM_001162914|    2|
|NM_001282866|    2|
|   NM_000746|    2|
|NM_001191323|    2|
|NM_001002266|    2|
|   NM_024618|    1|
|   NM_018899|    1|
+------------+-----+
only showing top 20 rows



For duplicate RefSeq's we will choose the value with the highest pLI value (i.e. most contrained) and where pLI is the same lowest oef rank (i.e. most constrained).

In [38]:
df_loft_ref.filter($"RefSeq" === "NM_206955" || $"RefSeq" === "NM_145021").show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+---------+-------+-------+------------+-----------------+----------------+---------+
|gnomadGeneSymbol|   gnomadGeneID|   TranscriptID|obs_mis|   exp_mis|    oe_mis|obs_syn|   exp_syn|    oe_syn|        p|      pLI|   pRec|  pNull|oe_lof_upper|oe_lof_upper_rank|oe_lof_upper_bin|   RefSeq|
+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+---------+-------+-------+------------+-----------------+----------------+---------+
|          MARCH8|ENSG00000165406|ENST00000319836|    124|1.6106e+02|7.6989e-01|     56|6.3569e+01|8.8094e-01|9.1457E-5|5.8689E-7|0.44685|0.55315|       1.314|            55585|               7|NM_145021|
|          MARCH8|ENSG00000165406|ENST00000395771|    124|1.6106e+02|7.6989e-01|     56|6.3569e+01|8.8094e-01|9.1457E-5|5.8689E-7|0.44685|0.55315|       1.314|            55587|   

In [39]:
val df_high_pLI = df_loft_ref.groupBy($"RefSeq").agg(max($"pLI"), min($"oe_lof_upper_rank"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_high_pLI: org.apache.spark.sql.DataFrame = [RefSeq: string, max(pLI): double ... 1 more field]


In [40]:
df_high_pLI.filter($"RefSeq" === "NM_206955" || $"RefSeq" === "NM_145021").show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+----------------------+
|   RefSeq| max(pLI)|min(oe_lof_upper_rank)|
+---------+---------+----------------------+
|NM_206955|1.3703E-6|                 52855|
|NM_145021|5.8689E-7|                 55585|
+---------+---------+----------------------+



Finally, create a table with unique RefSeq by joining to high pLI table

In [41]:
val df_loft_ref_uniq = ( df_loft_ref.join(df_high_pLI.as("pli"),
                                          df_loft_ref("RefSeq") === df_high_pLI("RefSeq") && 
                                          df_loft_ref("pLI") === df_high_pLI("max(pLI)") &&
                                          df_loft_ref("oe_lof_upper_rank") === df_high_pLI("min(oe_lof_upper_rank)"),
                                              "inner")
                         .drop($"pli.RefSeq").drop($"pli.max(pLI)").drop($"pli.min(oe_lof_upper_rank)") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_loft_ref_uniq: org.apache.spark.sql.DataFrame = [gnomadGeneSymbol: string, gnomadGeneID: string ... 15 more fields]


In [42]:
df_loft_ref_uniq.groupBy($"RefSeq").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      RefSeq|count|
+------------+-----+
|NM_001321544|    1|
|NM_001351544|    1|
|   NM_133371|    1|
|NM_001352944|    1|
|   NM_032801|    1|
|NM_001168272|    1|
|   NM_052939|    1|
|   NM_001446|    1|
|   NM_006032|    1|
|   NM_172357|    1|
|NM_001363944|    1|
|NM_001287756|    1|
|NM_001318239|    1|
|NM_001366319|    1|
|NM_001287221|    1|
|NM_001370357|    1|
|NM_001195055|    1|
|   NM_153230|    1|
|   NM_002562|    1|
|   NM_003696|    1|
+------------+-----+
only showing top 20 rows



In [43]:
df_loft_ref_uniq.orderBy(rand()).limit(10).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+----------+--------+---------+------------+-----------------+----------------+------------+
|gnomadGeneSymbol|   gnomadGeneID|   TranscriptID|obs_mis|   exp_mis|    oe_mis|obs_syn|   exp_syn|    oe_syn|        p|       pLI|    pRec|    pNull|oe_lof_upper|oe_lof_upper_rank|oe_lof_upper_bin|      RefSeq|
+----------------+---------------+---------------+-------+----------+----------+-------+----------+----------+---------+----------+--------+---------+------------+-----------------+----------------+------------+
|          ZNF276|ENSG00000158805|ENST00000443381|    439|3.4651e+02|1.2669e+00|    222|1.4997e+02|1.4803e+00|1.5906E-4| 7.8951E-6| 0.99617|0.0038225|       0.752|            29987|               3|NM_001113525|
|          HP1BP3|ENSG00000127483|ENST00000312239|    186|2.9258e+02|6.3572e-01|    125|1.1107e+02|1.1254e+00|3.5787E-5|   0.98422|0.015785|6.6087E-8|  

Constrained genes are those with a pLi > 0.9

In [44]:
df_loft_ref_uniq.filter($"pLi" >= 0.9).groupBy($"oe_lof_upper_bin").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|oe_lof_upper_bin|count|
+----------------+-----+
|               0| 7641|
|               1| 2603|
+----------------+-----+



Probability that transcript falls into distribution of unconstrained genes

In [45]:
df_loft_ref_uniq.filter($"pNull" <= 0.05).groupBy($"oe_lof_upper_bin").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|oe_lof_upper_bin|count|
+----------------+-----+
|               0| 7645|
|               1| 6082|
|               2| 6054|
|               3| 5360|
|               4| 3955|
|               5|  205|
|               6|   28|
+----------------+-----+



In [46]:
df_loft_ref_uniq.filter($"pNull" > 0.05).groupBy($"oe_lof_upper_bin").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|oe_lof_upper_bin|count|
+----------------+-----+
|               6| 4819|
|               5| 4639|
|               7| 4151|
|               8| 3094|
|               9| 2685|
|               4| 1348|
+----------------+-----+



 Probability that transcript falls into distribution of recessive genes (~46% o/e pLoF ratio; computed from gnomAD data)

In [47]:
df_loft_ref_uniq.filter($"pRec" <= 0.05).groupBy($"oe_lof_upper_bin").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|oe_lof_upper_bin|count|
+----------------+-----+
|               0| 7640|
|               1| 1527|
|               7|  980|
|               9|  798|
|               6|  756|
|               8|  728|
|               5|  256|
|               4|   37|
+----------------+-----+



In [48]:
df_loft_ref_uniq.filter($"pRec" > 0.05).groupBy($"oe_lof_upper_bin").count.sort($"count".desc).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|oe_lof_upper_bin|count|
+----------------+-----+
|               2| 6054|
|               3| 5360|
|               4| 5266|
|               5| 4588|
|               1| 4555|
|               6| 4091|
|               7| 3171|
|               8| 2366|
|               9| 1887|
|               0|    5|
+----------------+-----+



### Write out gnomAD pLI with RefSeq Metrics

In [50]:
(df_loft_ref_uniq.write.mode("overwrite")
                 .parquet("s3://nch-igm-research-projects/rna_stability/peter/gnomAD_pLI_RefSeq.parquet"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…